# Best Practices for Prompt Engineering



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION")    
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [2]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = '... \n\n ' #FIXME

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

This announcement highlights the ongoing collaboration between OpenAI and Microsoft, emphasizing their shared vision for developing advanced AI technology responsibly and safely. The multi-billion dollar investment from Microsoft will support OpenAI's independent research, aiding in the creation of AI that is beneficial to humanity. OpenAI's structure as a capped-profit company governed by a non-profit model reflects its commitment to prioritizing safety and equitable distribution of benefits, aligning with Microsoft's values. This partnership is seen as a crucial element in advancing their initiatives and achieving their mission.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [3]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In silicon dreams where ideas ignite,  
A spark of genius, an echo of light,  
OpenAI, a beacon so bright,  
In the vast realms of knowledge, taking flight.  

Born from the union of thought and machine,  
A canvas of code where visions convene,  
With language and logic, it weaves and it spins,  
A tapestry rich where learning begins.  

In the halls of innovation, collaboration thrives,  
With minds intertwined, the spirit alive,  
Pushing the boundaries, redefining the norm,  
A garden of wisdom, in every brainstorm.  

From texts to the tales that we long to convey,  
It helps us connect in a meaningful way,  
A partner in learning, a guide through the mist,  
In the age of AI, we find we can't resist.  

Empowering voices, from young ones to sage,  
Unlocking the stories, page after page,  
With ethics and foresight, it paves the new road,  
A journey of progress, where knowledge is sowed.  

So here's to the future, inspired and free,  
With OpenAI lighting the path we all see,  

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In a world of words and dreams,  
A dawn broke, soft and clear.  
Colors danced on digital canvas,  
Imagined visions brought near.

DALL-E, they named it, a child of light,  
With strokes of brilliance, pure and bold.  
In silence, it conjured what hearts desired,  
A tapestry woven from stories untold.

Art without borders, thought given form,  
In hands once weary, now vibrant and warm.  
The muse awakens, in pixels it gleams—  
A testament to the beauty of dreams.

So raise a glass to the sparks that ignite,  
To the canvas anew, where hope takes its flight.  
In the quiet of creation, let wonders unfurl,  
For in each drop of ink lies the pulse of our world.


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [5]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' ...(#FIXME)... \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}' #FIXME

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



{"text": "OpenAI", "start": 28, "end": 34}


In [6]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instructions = "Please extract the following types of entities from the text: \n" \
               "1. Company names.\n" \
               "2. Years.\n" \
               "3. Specific topics related to the content.\n" \
               "4. General and overarching topics.\n" \
               "The expected output format should include comma-separated lists for each type of entity."


prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



{"text": "OpenAI", "start": 28, "end": 34}


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [7]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, AI, safe, useful, powerful.


In [8]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctions = "Please extract keywords from the text and format the output as comma-separated lists."

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen ComfortCar Seat – a revolutionary car seat designed for ultimate safety and convenience. Engineered with advanced impact-absorbing materials and a state-of-the-art adjustable harness system, it ensures a secure fit for children of all ages. Featuring built-in smart sensors that monitor the seat's installation and provide real-time updates to your smartphone, the ComfortCar Seat combines cutting-edge technology with luxury. Experience peace of mind on every journey, knowing your child is protected and comfortable with this essential automotive upgrade.


In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the latest innovation in automotive safety and comfort: the SmartShield Car Seat. Engineered with advanced ergonomic design and adaptive memory foam, this car seat seamlessly molds to the contours of your body, providing unparalleled support on every journey. Integrated with smart technology, it features adjustable lumbar support, climate control, and built-in massage functions to enhance your driving experience. With its sleek and modern aesthetic, the SmartShield not only elevates your car's interior but also ensures maximum safety with its cutting-edge impact absorption system and easy installation. Transform the way you travel—experience luxury, safety, and comfort like never before.


# 7. Instead of just saying what not to do, say what to do instead

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you describe the issue you're encountering? Are you receiving any error messages?


In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging into your account. There are a few common issues that could be causing this problem. 

First, please ensure that you're entering the correct username and password. If you've forgotten your password, you can reset it by following the instructions in our help article here: www.samplewebsite.com/help/faq.

Additionally, check to see if your account might be locked due to multiple login attempts. If that's the case, waiting a few minutes before trying again might help. 

If you continue to experience issues, please refer to the help article for further troubleshooting steps. Let me know if there's anything else I can assist you with!


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [13]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = "Write a simple Python function that:\n" \
         "1. Asks for a number in miles.\n" \
         "2. Converts the miles to kilometers."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


Here's a simple Python function that accomplishes your request:

```python
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Enter the number of miles: "))
    
    # Convert miles to kilometers (1 mile = 1.60934 kilometers)
    kilometers = miles * 1.60934
    
    # Print the result
    print(f"{miles} miles is equal to {kilometers} kilometers.")

# Call the function
miles_to_kilometers()
```

You can run this code in a Python environment, and it will prompt you to enter a number of miles and then output the equivalent distance in kilometers.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [14]:

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": "OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro."}],
    max_tokens=400,
)

print(response.choices[0].message.content)



Así es, OpenAI y Microsoft han fortalecido su colaboración en el ámbito de la inteligencia artificial. La inversión sustancial de Microsoft se destina a potenciar la investigación y desarrollo de tecnologías de IA, lo que permitirá a OpenAI seguir avanzando en su misión de desarrollar inteligencia artificial segura y beneficiosa para la humanidad. A pesar de la colaboración con Microsoft, OpenAI se mantiene como una empresa de beneficios limitados, lo que significa que su estructura está diseñada para priorizar su misión social por encima de las ganancias económicas. Esta relación también ha facilitado que Microsoft integre las tecnologías de OpenAI en sus productos y servicios, aumentando la accesibilidad y aplicabilidad de estas innovaciones en el mercado.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant. response with the following structure: {'text': 'title', 'start': x, 'end': x}."},
                {"role":"user","content": "OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA." }],
    max_tokens=400,
)

print(response.choices[0].message.content)


{'text': 'OpenAI', 'start': 0, 'end': 6}


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [44]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)

In circuits bright, where visions blend,  
A partnership that knows no end,  
OpenAI, with dreams to soar,  
Joined hands with Microsoft, to explore.  

Together they weave a tapestry wide,  
Where human insight and machines collide,  
In realms of thought, creation takes flight,  
A dance of ideas, both bold and light.  

With every line of code they share,  
A spark ignites in digital air,  
Innovations bloom, new paths they chart,  
A symphony of intellect, a work of art.  

From text to vision, the future's call,  
In unity strong, they rise and enthrall,  
OpenAI and Microsoft, side by side,  
In the heart of progress, they abide.


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [56]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial  


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [65]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Introducing the ultimate tech backpack: equipped with a solar charger to keep your devices powered on the go, crafted from durable waterproof materials for all weather conditions, and designed with ergonomic comfort in mind. Perfect for tech enthusiasts and adventurers alike, this backpack seamlessly combines functionality and style. Explore the future of mobile convenience!
